# Session Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob

import json
import unittest

from common import Printable, projdir

from course import Course

from constants import *

## Session Class

Class to manage sessions

In [2]:
class Session(Printable):
    def __init__(self, event, dataPath, configPath, appConfig, verbosity=1):
        '''Initialise session object'''

        super().__init__(verbosity=verbosity)

        self.event = event
        self.entrants = event.entrants

        self.dataPath = dataPath
        self.configPath = configPath

        self.date = os.path.basename(configPath)
        self.year = self.date[:4]

        self.appConfig = appConfig

        self.courses = {}
        self.runs = {}
        self.numRuns = 0


    def loadConfig(self):
        '''Read config from JSON'''

        filename = os.path.join(self.configPath, COURSES_JSON)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            courseDict = json.loads(jsonTxt)
            
            for courseId in courseDict:
                # Course ID should be uppercase but just to be sure...
                courseId = courseId.upper()
                if courseId in self.courses:
                    raise ValueError('Duplicate course "{}" for {}'.format(courseId, self.date))

                course = Course(self, courseId,
                                courseDict[courseId]['Start Time'], courseDict[courseId]['End Time'],
                                self.appConfig, verbosity=self.verbosity)
                self.courses[courseId] = course


    def loadRuns(self, sailNos, gt31Ids):
        '''Read speeds from CSV files'''

        csvPaths = sorted(glob.glob(os.path.join(self.dataPath, '*')))
        
        for csvPath in csvPaths:
            prefix, sessionDate, courseId = os.path.splitext(os.path.basename(csvPath))[0].split('_')
            suffix = os.path.splitext(csvPath)[1]

            # Verify the filename
            if suffix.lower() != '.csv':
                raise ValueError('Invalid file suffix "{}" for {}'.format(suffix, os.path.basename(csvPath)))
            if prefix.upper() != 'GPSDATA':
                raise ValueError('Invalid file prefix "{}" for {}'.format(prefix, os.path.basename(csvPath)))
            if sessionDate != self.date:
                raise ValueError('Invalid file date "{}" for {}'.format(sessionDate, os.path.basename(csvPath)))
            if courseId.upper() not in self.courses:
                raise ValueError('Invalid course "{}" for {}'.format(courseId, os.path.basename(csvPath)))
            
            course = self.courses[courseId.upper()]
            course.loadRuns(csvPath, sailNos, gt31Ids)

        self.sortRuns()


    def storeRun(self, entrantId, speedRun):
        '''Store run in memory'''

        if entrantId in self.runs:
            self.runs[entrantId].append(speedRun)
        else:
            self.runs[entrantId] = [speedRun]
            
        self.numRuns += 1

        self.event.storeRun(entrantId, speedRun)


    def sortRuns(self):
        '''Sort runs for each person, fastest to slowest'''

        for entrantId in self.runs:
            self.runs[entrantId].sort(key=lambda x: x.data[T_SPEED], reverse=True)

## Unit Tests

A handful of very basic tests, including a dummy event class

In [3]:
class DummyEvent(Printable):
    def __init__(self):
        '''Initialise event object'''
        
        self.entrants = {0: None}

        self.runs = {}
        self.numRuns = 0


    def storeRun(self, entrantId, speedRun):
        
        if entrantId in self.runs:
            self.runs[entrantId].append(speedRun)
        else:
            self.runs[entrantId] = [speedRun]
            
        self.numRuns += 1

In [4]:
class TestSession(unittest.TestCase):
    '''Class to test Session class'''
    
    def test20191010(self, event=None, sailNos={}, gt31Ids={}):
        '''Test using 20191010 data'''

        if event is None:
            event = DummyEvent()

        sessionDate = '20191010'
        dataPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], GPSDATA_DIR, sessionDate)
        configPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], SESSIONS_DIR, sessionDate)
        
        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        session = Session(event, dataPath, configPath, appConfig, verbosity=0)
        session.loadConfig()
        session.loadRuns(sailNos, gt31Ids)

        self.assertEqual(session.numRuns, 638)
        self.assertEqual(len(session.runs), 40)

        # Check runs are sorted correctly
        for personId in session.runs:
            maxSpeed = 99.999
            for run in session.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def test20191011(self, event=None, sailNos={}, gt31Ids={}):
        '''Test using 20191011 data'''

        if event is None:
            event = DummyEvent()

        sessionDate = '20191011'
        dataPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], GPSDATA_DIR, sessionDate)
        configPath = os.path.join(projdir, EVENTS_DIR, sessionDate[:4], SESSIONS_DIR, sessionDate)
        
        # Vebosity is zero to suppress 'WARNING: Unrecognised GT-31 ID' 
        session = Session(event, dataPath, configPath, appConfig, verbosity=0)
        session.loadConfig()
        session.loadRuns(sailNos, gt31Ids)

        self.assertEqual(session.numRuns, 515)
        self.assertEqual(len(session.runs), 36)

        # Check runs are sorted correctly
        for personId in session.runs:
            maxSpeed = 99.999
            for run in session.runs[personId]:
                self.assertEqual(run.data[T_SPEED] <= maxSpeed, True)
                maxSpeed = run.data[T_SPEED]


    def test201910(self, event=None):
        '''Test using 20191010 + 20191011 data'''

        event = DummyEvent()
        sailNos = {}
        gt31Ids = {}

        # Load two sessions
        self.test20191010(event=event, sailNos=sailNos, gt31Ids=gt31Ids)
        self.test20191011(event=event, sailNos=sailNos, gt31Ids=gt31Ids)

        # Check the event totals
        self.assertEqual(event.numRuns, 1153)
        self.assertEqual(len(event.runs), 47)

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [5]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)

    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.068s

OK


## All Done!